# Using veo with Python module: `nlcpy.veo`

* [Py-veo by Examples](https://sx-aurora.github.io/posts/py-veo-by-examples/)
  - [py-veo/examples at master · SX-Aurora/py-veo](https://github.com/SX-Aurora/py-veo/tree/master/examples)

In [1]:
versioninfo()

Julia Version 1.7.3
Commit 742b9abb4d (2022-05-06 12:58 UTC)
Platform Info:
  OS: Linux (x86_64-redhat-linux)
  CPU: Intel(R) Xeon(R) Gold 6226 CPU @ 2.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, cascadelake)
Environment:
  JULIA_DEPOT_PATH = /home/manabu/.julia-1.7.3


---

In [2]:
#import Pkg; Pkg.add("PyCall")
using PyCall

In [3]:
using Pkg
Pkg.status("PyCall")

      Status `~/.julia-1.7.3/environments/v1.7/Project.toml`
  [438e738f] PyCall v1.93.1


In [4]:
#ENV["NMPI_LOCAL_RANK"]

In [5]:
veo = pyimport("nlcpy.veo")

PyObject <module 'nlcpy.veo' from '/opt/anaconda3/envs/jupyter/lib/python3.8/site-packages/nlcpy/veo/__init__.py'>

In [6]:
veo_proc = veo.VeoProc(1)

PyObject <nlcpy.veo._veo.VeoProc object at 0x7eff2404bf90>

In [8]:
veo_proc.nodeid

1

## Build

In [9]:
veo_build = veo.VeBuild()

PyObject <nlcpy.veo.vebuild.VeBuild object at 0x7eff289874c0>

In [32]:
veo_build.set_c_src("_test", raw"""
#include <stdio.h>
#include <unistd.h>
int flag = 0;
int do_sleep(int secs)
{
	
	printf("VE: flag value %d\n", flag);
        flag++;
	printf("VE: increased flag %d\n", flag);
	
	printf("VE: sleeping %d seconds\n", secs);
	sleep(secs);
	printf("VE: finished sleeping.\n");
	printf("VE: flag value %d\n", flag);
	return secs;
}
""")

In [33]:
veo_build._obj["_test"]._compiler, veo_build._obj["_test"]._flags

("/opt/nec/ve/bin/ncc", "-O2 -fpic -pthread -report-all -fdiag-vector=2")

In [37]:
veo_build.build_so(verbose=py"False")

"_test.so"

## Load

In [38]:
test_so = veo_proc.load_library(transcode(UInt8, "./_test.so"));

In [40]:
typeof(test_so)

PyObject

In [39]:
typeof(test_so.do_sleep)

PyObject

In [41]:
test_so.do_sleep

PyObject <VeoFunction object VE function b'do_sleep'None at 0x7eff1c631f40>

In [43]:
test_so.do_sleep.args_type("int")

In [44]:
ctx = veo_proc.open_context()

PyObject <nlcpy.veo._veo.VeoCtxt object at 0x7eff1c62ae00>

In [46]:
req = test_so.do_sleep(ctx, 5)
req.wait_result()

5